# Run the wavelength and the fringe frame maker notebooks first!

In [ ]:
import os
os.environ['OPENTSDB_PYTHON_METRICS_TEST_MODE'] = 'True'

In [ ]:
import importlib.resources
import requests
from banzai_floyds import settings
from banzai.utils.stage_utils import run_pipeline_stages
from banzai.utils.fits_utils import download_from_s3
import numpy as np
from astropy.io import ascii, fits
from astropy.coordinates import SkyCoord
from astropy import units
from matplotlib import pyplot
from glob import glob
import tarfile

In [ ]:
os.environ['DB_ADDRESS'] = 'sqlite:///test_data/test.db'
settings.processed_path= os.path.join(os.getcwd(), 'test_data')
settings.fpack=True
settings.db_address = os.environ['DB_ADDRESS']
settings.RAW_DATA_FRAME_URL = f'https://archive-api.lco.global/frames'

In [ ]:
# set up the context object.
import banzai.main
context = banzai.main.parse_args(settings, parse_system_args=False)

In [ ]:
from banzai_floyds import dbs

In [ ]:
dbs.ingest_standards(settings.db_address)

In [ ]:
# Start with just the profile.
context = banzai.main.parse_args(settings, parse_system_args=False)

In [ ]:
from banzai_floyds.tests.test_e2e import is_standard
test_data = ascii.read(os.path.join(importlib.resources.files('banzai_floyds.tests'), 'data/test_data.dat'))
for row in test_data:
    if 'e00.fits' in row['filename'] and is_standard(row['object']):
        run_pipeline_stages([{'filename': row['filename'], 'RLEVEL': 0, 'frameid': row['frameid']}], context)
for row in test_data:
    if 'e00.fits' in row['filename'] and not is_standard(row['object']):
        run_pipeline_stages([{'filename': row['filename'], 'RLEVEL': 0, 'frameid': row['frameid']}], context)

In [ ]:
def fits_header_to_wavelength(header):
    crval = float(header['CRVAL1'])
    crpix = float(header['CRPIX1'])
    # Convert crpix to be zero indexed
    crpix -= 1
    if 'CDELT1' in header.keys():
        cdelt = float(header['CDELT1'])
    else:
        cdelt = float(header['CD1_1'])
    npix = float(header['NAXIS1'])
    lam = np.arange(crval - cdelt * crpix ,
                    crval + cdelt * (npix - crpix) - 1e-4,
                    cdelt)
    return lam

In [ ]:
orders_to_color = {1: 'red', 2: 'blue'}

# Untar and extract the arc file
for filename in glob('test_data/*.tar.gz'):
    with tarfile.open(filename, 'r:gz') as f:
        for file_to_extract in f.getnames():
            if 'ttarc' == file_to_extract[:5]:
                f.extract(file_to_extract, os.path.join(os.getcwd(), 'test_data'))

tar_filenames = glob('test_data/*.tar.gz')

# plot the raw flux against the old pipeline's extractions
for row in test_data:
    if 'e00.fits' in row['filename'] and is_standard(row['object']):
        spectrum = glob(f'./test_data/*/*/*/processed/{row["filename"].replace("e00", "e91").replace(".fits", "-1d.fits")}')[0]
        hdu = fits.open(spectrum)
        site = row['filename'][:3]
        for tar_filename in tar_filenames:
            if os.path.basename(tar_filename)[:3].lower() == site:
                with tarfile.open(tar_filename, 'r:gz') as f:
                    for file_to_extract in f.getnames():
                        if 'ex.fits' in file_to_extract:
                            f.extract(file_to_extract, os.path.join(os.getcwd(), 'test_data'))
        for i in range(1, 3):
            in_order = hdu['EXTRACTED'].data['order'] == i
            data = hdu['EXTRACTED'].data
            data_to_plot = data['flux'][in_order]
            data_to_plot /= np.median(data_to_plot)
            pyplot.plot(data['wavelength'][in_order], data_to_plot)

            hdu[0].header['OBJECT'].upper()
            print(os.path.basename(spectrum))

            # get the old pipeline equivalent
            old_extraction = glob(f'test_data/tt{hdu[0].header["OBJECT"].upper()}*{orders_to_color[i]}*_ex.fits*')[0]
            old_hdu = fits.open(old_extraction)
            iraf_wavelengths = fits_header_to_wavelength(old_hdu[0].header)
            iraf_data_plot = old_hdu[0].data[0, :].T
            iraf_data_plot /= np.median(iraf_data_plot)
            pyplot.plot(iraf_wavelengths, old_hdu[0].data[0, :].T)
            pyplot.xlim(4800, 10200)
            pyplot.ylim(-0.1, 7)
            pyplot.show()
            pyplot.plot(hdu['TELLURIC'].data['wavelength'], hdu['TELLURIC'].data['telluric'])
            pyplot.show()

In [ ]:
from glob import glob
from astropy.io import fits

In [ ]:
extracted_spectra = glob('./test_data/*/*/*/processed/*-1d.fits.fz')

In [ ]:
#for f in extracted_spectra:
for f in extracted_spectra:
    hdu = fits.open(f)
    
    print(os.path.basename(f), hdu[0].header['OBJECT'])
    for order in [1, 2]:
        in_order = hdu['EXTRACTED'].data['order'] == order
        pyplot.plot(hdu['EXTRACTED'].data['wavelength'][in_order], hdu['EXTRACTED'].data['flux'][in_order], color='steelblue')
    pyplot.xlim(3300, 10200)
    pyplot.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

f = extracted_spectra[1]
hdu = fits.open(f)

fig = go.Figure(data={'x': hdu['SPECTRUM'].data['wavelength'], 'y': hdu['SPECTRUM'].data['flux']})
fig.show()

In [ ]:
import importlib.resources
for spectrum in extracted_spectra:
    if '0007' in spectrum or '0026' in spectrum:
        hdu = fits.open(spectrum)
        data = hdu['EXTRACTED'].data
        if 'bd' in hdu[0].header['OBJECT'].lower():
            standard_file = os.path.join(importlib.resources.files('banzai_floyds'), 'data', 'standards', 'bdp284211.fits')
        else:
            standard_file = os.path.join(importlib.resources.files('banzai_floyds'), 'data', 'standards', 'feige110.fits')
        standard_hdu = fits.open(standard_file)
        for order in [1, 2]:
            in_order = data['order'] == order
            pyplot.plot(data['wavelength'][in_order], data['flux'][in_order], color='steelblue')
        pyplot.plot(standard_hdu[1].data['wavelength'], standard_hdu[1].data['flux'], color='lightsalmon')
        pyplot.xlim(3000, 10500)
        pyplot.ylim(0, 2e-12)
        pyplot.show()


In [ ]:
#for f in extracted_spectra:
for f in extracted_spectra:
    hdu = fits.open(f)
    
    print(os.path.basename(f), hdu[0].header['OBJECT'])
    for order in [1, 2]:
        in_order = hdu['EXTRACTED'].data['order'] == order
        pyplot.plot(hdu['EXTRACTED'].data['wavelength'][in_order], hdu['EXTRACTED'].data['flux'][in_order], color='steelblue')
        pyplot.show()
        pyplot.plot(hdu['EXTRACTED'].data['wavelength'][in_order], hdu['EXTRACTED'].data['fluxraw'][in_order], color='steelblue')
        pyplot.show()
        pyplot.plot(hdu['EXTRACTED'].data['wavelength'][in_order], hdu['EXTRACTED'].data['background'][in_order], color='steelblue')
        pyplot.show()